In [1]:
import os
import json
import tensorflow as tf
from keras.utils.layer_utils import count_params
from model import get_model
from pathlib import Path

from data import get_dataset, get_zoo_elephants_images_and_labels, get_ELEP_images_and_labels
from train import SiameseModel
from metrics import get_kernel_mask, val, far, pairwise_accuracy

from tensorflow.keras import optimizers
from tensorflow.keras.applications.inception_v3 import InceptionV3

import model

In [2]:
from importlib import reload
reload(model)

<module 'model' from '/Users/deepakduggirala/Documents/project/siamese/model.py'>

In [3]:
with open('hyperparameters/initial_run.json', 'rb') as f:
    params = json.load(f)

In [4]:
data_dir = '/Users/deepakduggirala/Documents/ELPephant-cropped'
data_dir_2 = '/Users/deepakduggirala/Documents/Elephants-dataset-cropped-png-1024'

In [5]:
cache_files = {
        'train': str(Path(data_dir) / 'train.cache'),
        'val': str(Path(data_dir) / 'val.cache'),
        'val_2': str(Path(data_dir_2) / 'val.cache')
    }

In [22]:
train_ds, N_train = get_dataset(get_ELEP_images_and_labels, 
                                params, 
                                str(Path(data_dir)/'train'),
                                augment=True,  
                                cache_file=cache_files['train'], 
                                shuffle=True,
                                batch_size=128)

val_ds, N_val = get_dataset(get_ELEP_images_and_labels, 
                            params, 
                            str(Path(data_dir)/'val'),
                            augment=False, 
                            cache_file=cache_files['val'], 
                            shuffle=False, 
                            batch_size=32)

val_ds_2, N_val_2 = get_dataset(get_zoo_elephants_images_and_labels, 
                                params, 
                                data_dir_2,
                                augment=False, 
                                cache_file=cache_files['val_2'], 
                                shuffle=False, 
                                batch_size=32)

In [9]:
siamese_model = SiameseModel(params, True)
siamese_model.load_weights('/Users/deepakduggirala/Downloads/best_weights')

In [12]:
siamese_model.compile(optimizer=optimizers.Adam(learning_rate=params['lr']))

In [34]:
# https://stackoverflow.com/a/47738812/2580077

class AdditionalValidationSets(tf.keras.callbacks.Callback):
    def __init__(self, validation_sets, verbose=0):
        super(AdditionalValidationSets, self).__init__()
        self.validation_sets = validation_sets
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs=None):

        # evaluate on the additional validation sets
        for validation_set in self.validation_sets:
            val_ds, validation_set_name = validation_set

            results = self.model.evaluate(val_ds,
                                          verbose=self.verbose,
                                          return_dict=True)

            for metric, result in results.items():
                valuename = validation_set_name + '_' + metric
                logs[valuename]=result

In [32]:
history = AdditionalValidationSets([(val_ds_2.take(2), 'val_2')], verbose=0)

In [ ]:
results = siamese_model.evaluate(val_ds, verbose=1, return_dict=True)

In [18]:
results.items()

dict_items([('loss', 0.06534186750650406), ('VAL', 0.981441080570221), ('FAR', 0.3229336440563202)])

In [33]:
siamese_model.fit(train_ds.take(2),
                      epochs=2,
                      validation_data=val_ds.take(2),
                      callbacks=[history])

Epoch 1/2
2/2 [==============================] - 18s 11s/step - loss: 4.1047 - VAL: 1.0000 - FAR: 0.3333 - val_loss: 0.0491 - val_VAL: 0.9576 - val_FAR: 0.3338 - val_2_loss: 0.0000e+00 - val_2_VAL: 0.9909 - val_2_FAR: nan
Epoch 2/2
2/2 [==============================] - 20s 12s/step - loss: 3.9832 - VAL: 0.9700 - FAR: 0.3328 - val_loss: 0.0473 - val_VAL: 0.9576 - val_FAR: 0.3179 - val_2_loss: 0.0000e+00 - val_2_VAL: 0.9909 - val_2_FAR: nan


In [36]:
import numpy as np

In [39]:
np.arange(0.01,0.105,0.005)

array([0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 , 0.045, 0.05 ,
       0.055, 0.06 , 0.065, 0.07 , 0.075, 0.08 , 0.085, 0.09 , 0.095,
       0.1  ])